In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
with open('X.npy', 'rb') as f:
  X = np.load(f)

with open('y.npy', 'rb') as f:
  y = np.load(f)

In [6]:
X.shape, y.shape

((1460, 60), (1460, 1))

In [7]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [8]:
# K-fold cross validation
K = 10
kf = KFold(n_splits=K)

In [11]:
rmses = []
for train_index, test_index in kf.split(scaled_X):
  scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  # training
  model = keras.Sequential(
      [
        keras.Input(shape=scaled_X_train.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
      ]
  )

  # model.compile(loss="mse", optimizer="adam")

  opt = keras.optimizers.Adam(learning_rate=0.005)
  model.compile(loss="mse", optimizer=opt)

  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
  model.fit(scaled_X_train, scaled_y_train,
                    batch_size=1, epochs=150,
                    callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

  # evaluation
  pred = model.predict(scaled_X_test).reshape((-1, 1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

  print(rmse)
  print("--------------")
  

  rmses.append(rmse)

print("average rmse:", np.mean(rmses))

Epoch 1/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0073 - val_loss: 0.0027
Epoch 2/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0048 - val_loss: 0.0022
Epoch 3/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0039 - val_loss: 0.0144
Epoch 4/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0046 - val_loss: 0.0017
Epoch 5/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0034 - val_loss: 0.0028
Epoch 6/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0035 - val_loss: 0.0017
Epoch 7/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0019
Epoch 8/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0023 - val_loss: 0.0022
Epoch 9/150
1248/1248 [==============================] - 1s 1ms/step - loss: 0.0029 - val_loss: 0.0045
Epoch 10/150
1248/1248 [==============================] - 1s 1ms/step - l

# Hyperparameter
- batch size
- learning rate
- number of layers
- feature sizes
- activation functions
- optimizers

# Grid Search

In [24]:
batch_sizes = np.arange(5, 10, 2) # batch size는 5에서부터 10까지 2칸씩 띄어가면서 탐색하고 싶음.

In [25]:
batch_sizes

array([5, 7, 9])

In [26]:
learning_rates = [0.005, 0.01, 0.02]  # learning rate는 0.005, 0.01, 0.02를 탐색할 것

In [27]:
# 총 9개의 세트가 나온다.
# 여기서 최적의 rmse값이 나오는 hyperparameter set을 골라내면 hyperparameter tuning을 했다고 할 수 있다.
for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print("batch_size:", batch_size, "learning_rate:", learning_rate)

batch_size: 5 learning_rate: 0.005
batch_size: 5 learning_rate: 0.01
batch_size: 5 learning_rate: 0.02
batch_size: 7 learning_rate: 0.005
batch_size: 7 learning_rate: 0.01
batch_size: 7 learning_rate: 0.02
batch_size: 9 learning_rate: 0.005
batch_size: 9 learning_rate: 0.01
batch_size: 9 learning_rate: 0.02


In [28]:
results = []
for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print(batch_size, learning_rate)
    for train_index, test_index in kf.split(scaled_X):
      scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
      scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
      y_test = y[test_index]

      # training
      model = keras.Sequential(
          [
            layers.InputLayer(input_shape=scaled_X_train.shape[-1]),
            layers.Dense(96, activation='relu'),
            layers.Dense(48, activation='relu'),
            layers.Dense(1)
          ]
      )

      # model.compile(loss="mse", optimizer="adam")

      opt = keras.optimizers.Adam(learning_rate=0.005)
      model.compile(loss="mse", optimizer=opt)

      early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
      model.fit(scaled_X_train, scaled_y_train,
                        batch_size=1, epochs=150,
                        callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

      # evaluation
      pred = model.predict(scaled_X_test).reshape((-1, 1))
      pred = y_min_max_scaler.inverse_transform(pred)
      rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

      print(rmse)
      print("--------------")

      result = {}
      result['batch_size'] = batch_size
      result['learning_rate'] = learning_rate
      result['rmse'] = rmse

      results.append(result)
      break
  break

5 0.005
Epoch 1/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0069 - val_loss: 0.0020
Epoch 2/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0045 - val_loss: 0.0016
Epoch 3/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0043 - val_loss: 0.0020
Epoch 4/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0043 - val_loss: 0.0020
Epoch 5/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0030 - val_loss: 0.0068
Epoch 6/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0028 - val_loss: 0.0017
Epoch 7/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0032 - val_loss: 0.0018
Epoch 8/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0024 - val_loss: 0.0018
Epoch 9/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0021 - val_loss: 0.0104
71074.86494270364
--------------
5 0.01
Epoch 1/150
1248/1248 [==

In [29]:
results

[{'batch_size': 5, 'learning_rate': 0.005, 'rmse': 71074.86494270364},
 {'batch_size': 5, 'learning_rate': 0.01, 'rmse': 24298.06777689446},
 {'batch_size': 5, 'learning_rate': 0.02, 'rmse': 36822.160466083864}]

# Random Search

In [18]:
# grid search에서는 범위를 어떤 룰에 의해 만들었다면, 
# random search는 시작과 끝점만 정해주고, 그 사이에서 랜덤으로 값을 뽑아오게 된다.
import random

In [19]:
batch_sizes = random.sample(range(1, 11), 5)

In [20]:
batch_sizes

[9, 5, 7, 6, 1]

In [21]:
learning_rates = np.random.uniform(low=0.005, high=0.1, size=(4,))

In [22]:
# 정말 상상도 못했던 숫자에서 최적의 값이 나올 수 있음!
learning_rates

array([0.0122374 , 0.08654849, 0.037508  , 0.09966664])